# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных:
- children — количество детей в семье;
- days_employed — общий трудовой стаж в днях;
- dob_years — возраст клиента в годах;
- education — уровень образования клиента;
- education_id — идентификатор уровня образования;
- family_status — семейное положение;
- family_status_id — идентификатор семейного положения;
- gender — пол клиента;
- income_type — тип занятости;
- debt — имел ли задолженность по возврату кредитов;
- total_income — ежемесячный доход;
- purpose — цель получения кредита.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from collections import Counter

In [2]:
df = pd.read_csv('/datasets/data.csv')

In [3]:
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В столбцах 'days_employed' и 'total_income' встречаются пропущенные значения.
В столбце 'education' - разный регистр.
В столбце 'purpose' одни и те же цели написаны по-разному.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Для удобства вещественный тип данных заменим на целочисленный на втором шаге.

В столбцах 'days_employed' и 'total_income' одинаковое количество пропусков, возможно, что они в одних и тех же строках.
Проверим это.

In [5]:
df[(df['days_employed'].isnull() == True) & (df['total_income'].isnull() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Действительно, в 2174 строках пропущены значения в двух столбцах одновременно.

In [6]:
df.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Процент пропущенных значений - 10%.

In [7]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


В столбцах 'children' и 'days_employed' минимальное и максимальное значения настораживают.
В столбце 'dob_years' минимальный возраст равен 0. Скорее всего, данные не были заполнены.
Проверим количество некорректных значений.

In [8]:
df.groupby('children')['children'].count()

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

In [9]:
df.groupby('dob_years')['dob_years'].count().head()

dob_years
0     101
19     14
20     51
21    111
22    183
Name: dob_years, dtype: int64

Количество строк с количеством детей '-1' и '20' и с количеством людей без возраста - небольшое в данной выборке. 

**Вывод**

1) Были найдены пропуски в столбцах 'days_employed' и 'total_income' и эти пропуски взаимосвязаны. Причиной может быть ошибка при выгрузке данных или если информация была взята из разных источников. Доля таких пропусков высока (10%), поэтому на следующем шаге заполним эти пропуски средним значением или медианой.

2) Данные в столбце 'days_employed' бывают отрицательными и сильно завышены. Для целей исследования они не нужны, поэтому рассматривать их подробно не будем.

3) В столбцах 'days_employed' и 'total_income' тип данных вещественный, для удобства использования заменим его на целочисленный на втором шаге.

4) В столбце 'education' разные регистры букв, при поиске и удалении дубликатов значения этого столбца приведем к нижнему регистру.

5) В столбце 'purpose' выявлены похожие цели, на следующем шаге с помощью лемматизации и категоризации сократим количество целей.

6) В столбцах 'children' и 'dob_years' выявлены аномалии, хоть их количество и небольшое, но на следующем шаге заменим их.

## Шаг 2. Предобработка данных

### Обработка пропусков и аномальных значений

In [10]:
df.pivot_table(index='income_type', values = 'total_income', aggfunc={'median', 'mean'})

,mean,median
income_type,,
безработный,131339.751676,131339.751676
в декрете,53829.130729,53829.130729
госслужащий,170898.309923,150447.935283
компаньон,202417.461462,172357.950966
пенсионер,137127.465690,118514.486412
предприниматель,499163.144947,499163.144947
сотрудник,161380.260488,142594.396847
студент,98201.625314,98201.625314


Найдём среднюю и медиану для столбца 'total_income' по каждому типу занятости.
В некоторых случая они равны.

In [11]:
df.loc[df['total_income'].isnull(), 'income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Найдём, у каких клиентов пропущены значения в столбце 'total_income'.

In [12]:
df['total_income'] = df.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median()))

В данном случае пропуски заполнены медианой, чтобы не завышать информацию о доходе заёмщиков.

In [13]:
df.groupby('gender')['gender'].count()

gender
F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В ходе анализа распределения клиентов по полу был найден третий пол - XNA. Это ошибочное заполнение, возможно, не был указан пол и вовсе. Т.к. всего лишь 1 такое значение, то его можно или удалить, или заполнить любым полом.
Более вероятно, что это клиент женского пола, поэтому заменим значение на 'F'.

In [14]:
df['gender'] = df['gender'].replace('XNA', 'F')
df.groupby('gender')['gender'].count()

gender
F    14237
M     7288
Name: gender, dtype: int64

Третий пол убран.

In [15]:
df['dob_years'] = df['dob_years'].replace(0, df['dob_years'].mean())

In [16]:
df['dob_years'].describe()

count    21525.000000
mean        43.496522
std         12.218174
min         19.000000
25%         34.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Нулевой возраст заменили на средний. Теперь заёмщиков без возраста нет.

In [17]:
def age_group(dob_years):
        """
        Возвращает возрастную группу по значению возраста, используя правила:
        - 'молодежь' - <= 35 лет;
        - 'средний возраст' - 36-50 лет;
        - 'зрелый возраст' - 51 год и более.
        """

        if dob_years <= 35:
                return 'молодежь'
        if dob_years <= 50:
                return 'средний возраст'
        return 'зрелый возраст' 
df['age_group'] = df['dob_years'].apply(age_group)
df['age_group'].value_counts()

средний возраст    8249
зрелый возраст     6682
молодежь           6594
Name: age_group, dtype: int64

Создадим возрастные категории.

In [18]:
df['days_employed'] = df['days_employed'].abs()

In [19]:
df['years_employed'] = df['days_employed'] / 365

In [20]:
df.pivot_table(index='income_type', values = 'days_employed', aggfunc={'count', 'min', 'max', 'median', 'mean'})

,count,max,mean,median,min
income_type,,,,,
безработный,2,395302.838654,366413.652744,366413.652744,337524.466835
в декрете,1,3296.759962,3296.759962,3296.759962,3296.759962
госслужащий,1312,15193.032201,3399.896902,2689.368353,39.954170
компаньон,4577,17615.563266,2111.524398,1547.382223,30.195337
пенсионер,3443,401755.400475,365003.491245,365213.306266,328728.720605
предприниматель,1,520.848083,520.848083,520.848083,520.848083
сотрудник,10014,18388.949901,2326.499216,1574.202821,24.141633
студент,1,578.751554,578.751554,578.751554,578.751554


In [21]:
df.pivot_table(index='income_type', values = 'years_employed', aggfunc={'count', 'min', 'max', 'median', 'mean'})

,count,max,mean,median,min
income_type,,,,,
безработный,2,1083.021476,1003.873021,1003.873021,924.724567
в декрете,1,9.032219,9.032219,9.032219,9.032219
госслужащий,1312,41.624746,9.314786,7.368132,0.109463
компаньон,4577,48.261817,5.784998,4.239403,0.082727
пенсионер,3443,1100.699727,1000.009565,1000.584401,900.626632
предприниматель,1,1.426981,1.426981,1.426981,1.426981
сотрудник,10014,50.380685,6.373970,4.312884,0.066141
студент,1,1.585621,1.585621,1.585621,1.585621


Отрицательные значения в столбце 'days_employed' возникли, скорее всего, по ошибке данных - лишнее тире. Чтобы избавиться от таких чисел, воспользуемся методом abs(), тогда модуль каждого значения даст нам положительные числа.
Для удобства анализа создаём новый столбец 'years_employed', в котором стаж в днях переведён в стаж в годах.
Далее смотрим различные значения стажа по каждого типу занятости.
Видим, аномальные значения в группах - 'безработный' и 'пенсионер'. В них минимальные значения стажа - от 900 лет, что совершенно нереально и таким данным невозможно доверять, причём они однородны в этих группах, что говорит об ошибках во всей группе. Предлагаю такие данные удалить совсем и заполнить их, исходя из данных стажа по возрастной категории и пола.

В группах 'госслужащий', 'компаньон', 'сотрудник' максимальные значения достаточно велики, но они в рамках физически возможного стажа работы либо у них были какие-то коэффицинты на прибавку к стажу (два года стажа за один год работы и т.п.).

In [22]:
df.loc[df['days_employed'] > 328500, 'days_employed'] = np.nan

Заменили очень большие значения стажа на Nan.

In [23]:
df = df.drop('years_employed', axis=1)

Удалили ранее созданный столбец.

In [24]:
df.pivot_table(index={'gender', 'age_group'}, values = 'days_employed', aggfunc={'count', 'min', 'max', 'median', 'mean'})

count           max         mean       median  \
age_group       gender                                                  
зрелый возраст  F        1808  18388.949901  3689.763401  2586.464532   
                M         960  15267.541183  2825.746632  1929.018754   
молодежь        F        3473   7622.710756  1571.666422  1263.981419   
                M        2461   7375.209318  1493.875508  1120.131612   
средний возраст F        4665  13724.223884  2770.877741  2095.473352   
                M        2539  11787.798590  2356.160949  1732.406585   

                              min  
age_group       gender             
зрелый возраст  F       58.276481  
                M       65.163190  
молодежь        F       24.141633  
                M       24.240695  
средний возраст F       37.726602  
                M       30.195337

Посмотрели различные значения стажа в разрезе пола и возрастной категории. У женщин всех категорий стаж работы больше, чем у мужчин. Это просто наблюдение.
Для заполнения пропусков в стаже возьмём медиану, чтобы не завышать данные.

In [25]:
df['days_employed'] = df['days_employed'].fillna(df.groupby(['gender','age_group'])['days_employed'].transform('median'))

In [26]:
df['days_employed'].describe()

count    21525.000000
mean      2320.039371
std       1996.310462
min         24.141633
25%       1025.608174
50%       1960.157814
75%       2586.464532
max      18388.949901
Name: days_employed, dtype: float64

Теперь максимальный стаж равен 18388,95 дням или 50,38 годам, что же больше похоже на правду.

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
age_group           21525 non-null object
dtypes: float64(3), int64(4), object(6)
memory usage: 2.1+ MB


Пропуски в столбцах 'days_employed' и 'total_income' больше не обнаружены.

In [28]:
df['children'] = df['children'].replace(-1, 1)

In [29]:
df['children'] = df['children'].replace(20, 2)

In [30]:
df['children'].describe()

count    21525.000000
mean         0.479721
std          0.755528
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          5.000000
Name: children, dtype: float64

Аномальные значения '-1' и '20' заменили на '1' и '2'. Больше они не обнаружены.

**Вывод**

1) Причина возникновения пропусков в доходах и стаже работы заёмщиков - некорректное заполнение информации либо источники информации разные и при их объединении часть информации потерялась. 
Пропуски в доходах заполнены медианой в зависимости от типа занятости.
Пропуски в стаже заполнены медианой в зависимости от пола и возрастной группы.

2) Причина нулевого возраста - ошибки при заполении/обработке данных о дате рождения, соответственно, невозможно было посчитать возраст клиента. Такие данные заменены на средний возраст.

3) Причина аномалий в количестве детей - ошибки/опечатки при заполнении данных сотрудниками/клиентами банка. Предположили, что '-1' - это 1 ребёнок, а '20' - это 2 ребёнка.

### Замена типа данных

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
age_group           21525 non-null object
dtypes: float64(3), int64(4), object(6)
memory usage: 2.1+ MB


После обработки столбца 'dob_years' тип данных поменялся на вещественный.
Заменим тип данных в столбцах 'dob_years', 'days_employed' и 'total_income'

In [32]:
df['dob_years'] = df['dob_years'].astype('int') 

In [33]:
df['days_employed'] = df['days_employed'].astype('int') 

In [34]:
df['total_income'] = df['total_income'].astype('int') 

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
age_group           21525 non-null object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


**Вывод**

В столбцах 'dob_years', 'days_employed' и 'total_income' вещественный тип данных заменён на целочисленный методом astype() для удобства использования в анализе. Точность данных после '.' в данном случае избыточна.

### Обработка дубликатов

In [36]:
df.duplicated().sum()

54

Количество дубликатов - 54. Но их количество может увеличиться после приведения данных в столбце 'education' к нижнему регистру.

In [37]:
df['education'] = df['education'].str.lower()

In [38]:
df.duplicated().sum()

71

Количество дубликатов возросло до 71.

In [39]:
df = df.drop_duplicates().reset_index(drop=True)

In [40]:
df.duplicated().sum()

0

**Вывод**

Поиск дубликатов методом duplicated() в сочетании с методом sum() позволил получить количество дубликатов - 54.
Но в столбце 'education' были символы с разными регистрами, поэтому они были приведены к нижнему регистру методом str.lower().
Таким образом, количество дубликатов увеличилось до 71.
Далее для удаления дубликатов использован метод для удаления повторяющихся строк drop_duplicates() в сочетании с методом reset_index, который перезаписывает индексы без пропусков. 
В конце сделана проверка на отсутствие дубликатов.

Причина появления дубликатов - ошибочное создание данных сотрудниками или информация была взята из разных источников.
Дубликаты могут исказить общую картину, поэтому их удаление оправдано, хоть и их количество небольшое.

### Лемматизация

In [41]:
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Получены уникальные значения целей кредита, их достаточно много, но встречаются однотипные.
Разделим на 4 группы:
- жилье/недвижимость;
- автомобиль;
- свадьба;
- образование.

In [42]:
m = Mystem()

purpose_unique = df['purpose'].unique()
lemmas = []
for string in purpose_unique:
    lemma = m.lemmatize(string)
    lemmas.append(lemma)

Лемматизация списка целей кредита с помощью pymystem3.

In [43]:
df['purpose_lemmatize'] = df['purpose'].apply(m.lemmatize)

# функция для изменения цели кредита на одну из 4 групп
def change_purpose(purpose):
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if 'образование' in purpose:
        return 'образование'
    if 'свадьба' in purpose:
        return 'свадьба'
    return 'недвижимость'
# создание нового столбца с присвоением группы
df['purpose_group'] = df['purpose_lemmatize'].apply(change_purpose)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_lemmatize,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,средний возраст,"[покупка, , жилье, \n]",недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,средний возраст,"[приобретение, , автомобиль, \n]",автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,молодежь,"[покупка, , жилье, \n]",недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,молодежь,"[дополнительный, , образование, \n]",образование
4,0,2586,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,зрелый возраст,"[сыграть, , свадьба, \n]",свадьба


С помощью функции change_purpose цели кредита были поделены на 4 группы.

In [44]:
df['purpose_group'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_group, dtype: int64

**Вывод**

С начала были получены уникальные значения в столбце 'purpose', выделены 4 большие группы (недвижимость, автомобиль, образование и свадьба). Затем проведена лемматизация целей кредита с помощью библиотеки pymystem3, создана функция для присвоения большой группы и создан новый столбец с обработанными целями кредита.
Распределение по группам неравномерное - количество кредитов, выданных для недвижимости, значительно выше, чем для остальных целей.

### Категоризация данных

In [45]:
education_dict = df[['education_id','education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Для данного набора данных можно выделить словарь - уровень образования - id уровня образования.

In [46]:
family_status_dict = df[['family_status_id','family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Для данного набора данных также можно выделить словарь - семейное положение - id семейного положения.

In [47]:
df['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Т.к. многодетных заёмщиков немного (3 и более детей), то не будем выделять их в отдельную группу.
Сделаем две группы - без детей и с детьми.

In [48]:
def children_group(children):
        """
        Возвращает группу по значению количества детей, используя правила:
        - детей нет - 0 детей;
        - дети есть - 1 и более детей.
        """

        if children == 0:
                return 'детей нет'        
        return 'дети есть' 
df['children_group'] = df['children'].apply(children_group)

In [49]:
df['children_group'].value_counts()

детей нет    14091
дети есть     7363
Name: children_group, dtype: int64

In [50]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [51]:
df['total_income'].describe()

count     21454.000
mean     165319.572
std       98187.303
min       20667.000
25%      107623.000
50%      142594.000
75%      195820.250
max     2265604.000
Name: total_income, dtype: float64

Создадим 4 группы, равные группы по доходам, используя квартили - 25%, 50%, 75%.

In [52]:
def income_group(total_income):
        """
        Возвращает группу по значению дохода, используя правила:
        - '1' - <= 107623;
        - '2' - 107623-142594;
        - '3' - 142594-195820;
        - '4' - более 195820.
        """

        if total_income <= 107623:
                return '1'
        if total_income <= 142594:
                return '2'
        if total_income <= 195820:
                return '3'
        return '4' 
df['income_group'] = df['total_income'].apply(income_group)

In [53]:
df['income_group'].value_counts()

2    5479
4    5364
1    5364
3    5247
Name: income_group, dtype: int64

**Вывод**

Были выделены следующие категории, необходимые для ответов на вопросы в следующем шаге:

по семейному статусу:
- женат / замужем;
- гражданский брак;
- вдовец / вдова;
- в разводе;
- не женат / не замужем.

по количеству детей:
- детей нет;
- дети есть.

Сделано две группы, т.к. группа многодетные, оказалась, небольшой.

по уровню дохода:
- '1' - до 107623;
- '2' - 107623-142594;
- '3' - 142594-195820;
- '4' - более 195820.

Сделано 4 группы, примерно равные по количеству.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [54]:
# используем сводную таблицу и расчётный столбец 'ratio_children' = клиенты с долгами / на всех клиентов
df_pivot = df.pivot_table(index=['children_group', 'children'], columns = 'debt', values = 'gender', aggfunc='count') 
df_pivot['ratio_children'] = df_pivot[1] / (df_pivot[0] + df_pivot[1]) * 100
#df_pivot.sort_values('ratio_children', ascending = False)
df_pivot

debt                            0        1  ratio_children
children_group children                                   
детей нет      0        13028.000 1063.000           7.544
дети есть      1         4410.000  445.000           9.166
               2         1926.000  202.000           9.492
               3          303.000   27.000           8.182
               4           37.000    4.000           9.756
               5            9.000      nan             nan

**Вывод**

Доля клиентов с задолженностью и детьми - 9,21%, а без детей - 7,54%.
На основе полученных данных можно сделать вывод, что при наличии детей клиенты банка платят по кредитам немного хуже.
Возможная причина - важность удовлетворения потребностей своих детей выше, чем вовремя оплаченный взнос по кредиту.

При детальном рассмотрении зависимости количества детей на возврат кредитов было выявлено, что с ростом количества детей увеличивается доля должников, хотя есть исключение - семьи с 4 детьми возвращают кредиты лучше, чем остальные, а с 5 - никогда не были должниками (но выборка очень маленькая - всего лишь 9 клиентов).


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [55]:
# используем сводную таблицу и расчётный столбец 'ratio_status' = клиенты с долгами / на всех клиентов
df_pivot = df.pivot_table(index='family_status', columns = 'debt', values = 'family_status_id', aggfunc='count') 
df_pivot['ratio_status'] = df_pivot[1] / (df_pivot[0] + df_pivot[1]) * 100
df_pivot.sort_values('ratio_status', ascending = False)

debt,0,1,ratio_status
family_status,,,
Не женат / не замужем,2536,274,9.751
гражданский брак,3763,388,9.347
женат / замужем,11408,931,7.545
в разводе,1110,85,7.113
вдовец / вдова,896,63,6.569


**Вывод**

При отсутствии официального супруга/супруги (первые две категории) доля клиентов с задолженностью примерно одинаковая (9,75% и 9,35%) и эти доли выше, чему у клиентов, у которых есть официальный брак или был ранее (их доли должников также примерно равны 7,55%, 7,11% и 6,57%)

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [56]:
# используем сводную таблицу и расчётный столбец 'ratio_income' = клиенты с долгами / на всех клиентов
df_pivot = df.pivot_table(index='income_group', columns = 'debt', values = 'family_status_id', aggfunc='count') 
df_pivot['ratio_income'] = df_pivot[1] / (df_pivot[0] + df_pivot[1]) * 100
df_pivot.sort_values('ratio_income', ascending = False)

debt,0,1,ratio_income
income_group,,,
2,4996,483,8.815
3,4799,448,8.538
1,4937,427,7.960
4,4981,383,7.140


**Вывод**

Группы заёмщиков по уровню дохода:
- '1' - <= 107623;
- '2' - 107623-142594;
- '3' - 142594-195820;
- '4' - более 195820.

Наименьшая доля должников по кредитам оказалась в группе 4 - самых обеспеченных людей (7,14%), а следом за ними, наоборот, группа 1 - наименее обеспеченных (7,96%).
Это может быть связано с тем, что обеспеченные люди берут кредиты в тех случаях, когда уверены, что смогут платить по ним платежи. Тоже самое и с наименее обеспеченными - такие люди лучше планируют свои расходы, в первую очередь обязательные платежи (например, по кредиту).

Наибольшая доля должников в средних группах - 2 и 3, такие люди, скорее всего, не совсем правильно планируют свои расходы. 

- Как разные цели кредита влияют на его возврат в срок?

In [57]:
# используем сводную таблицу и расчётный столбец 'ratio_purpose' = клиенты с долгами / на всех клиентов
df_pivot = df.pivot_table(index='purpose_group', columns = 'debt', values = 'family_status_id', aggfunc='count') 
df_pivot['ratio_purpose'] = df_pivot[1] / (df_pivot[0] + df_pivot[1]) * 100
df_pivot.sort_values('ratio_purpose', ascending = False)

debt,0,1,ratio_purpose
purpose_group,,,
автомобиль,3903,403,9.359
образование,3643,370,9.220
свадьба,2138,186,8.003
недвижимость,10029,782,7.233


**Вывод**

Наибольшая доля должников среди клиентов, которые брали кредит для приобретения автомобиля (9,36%) и получения образования (9,22%).
Наименьшая доля - среди тех, кто указал цель кредита - недвижимость (7,23%).
Промежуточное значение заняла категория заёмщиков, которые брали креди для организации свадьбы (8%).

Причина: приобретение недвижимости - более затратно, чем другие расходы. Соответственно, платежи при просрочках будут ощутимо выше, чем в других случаях.

## Шаг 4. Общий вывод

Исходя из полученных данных, можно сделать следующий вывод.
Самые лучшие заёмщики - это люди со следующими параметрами:
- без детей (7,54%);
- есть или был официальный брак (7,55%);
- с доходом более 195820 (7,14%);
- указывающие цель кредита - недвижимость (7,23%).

Самые худшие заёмщики - это люди со следующими параметрами:
- с детьми (9,21%);
- не состоящие в официальном браке (9,75%);
- с доходом от 107623 и до 142594 (8,82%);
- указывающие цель кредита - автомобиль (9,36%).